# p24+ cells

In [ ]:
import os
import pandas as pd
import numpy as np
import skimage.io as io
import xarray as xr

In [ ]:
cell_tab_path = "../data/tables/cell_table_size_normalized.csv"
nimbus_tab_path = "../data/tables/nimbus_output.csv"
dist_mat_dir = "../data/spatial_analysis/dist_mats"
seg_dir = "../data/segmentation_masks"
p24_mask_dir = "../data/region_masks/p24"

## Get p24+ cells

In [ ]:
cell_tab = pd.read_csv(cell_tab_path)
nimbus_tab = pd.read_csv(nimbus_tab_path)
combined_tab = pd.merge(cell_tab[['fov','label','cell_meta_cluster','in_follicle_mask']], nimbus_tab, on=['fov','label'])

In [ ]:
# only keep cells above a threshold and inside of manually created p24 masks
nimbus_thresh = 0.4
keep_fovs = os.listdir(p24_mask_dir)
keep_fovs = [x.replace(".tiff","") for x in keep_fovs]

all_cells = pd.DataFrame(columns=['fov','label'])
for fov in keep_fovs:
    seg_mask = io.imread(os.path.join(seg_dir, fov+".tiff"))
    p24_mask = io.imread(os.path.join(p24_mask_dir, fov+".tiff"))
    keep_labels = np.unique(seg_mask[p24_mask==1])
    
    subsetted_tab = combined_tab[(combined_tab["fov"]==fov) & (combined_tab["label"].isin(keep_labels)) & (combined_tab["HIV1p24"] > nimbus_thresh)]
    all_cells = pd.concat([all_cells, subsetted_tab[['fov','label','cell_meta_cluster','in_follicle_mask']]])

all_cells.to_csv("../data/p24pos_cells.csv", index=False)

## Make masks

In [ ]:
out_dir = "../data/region_masks/p24pos_cells"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

p24pos_cell_tab = pd.read_csv("../data/p24pos_cells.csv")

all_fovs = np.unique(p24pos_cell_tab['fov'])
for fov in all_fovs:
    keep_labs = p24pos_cell_tab[p24pos_cell_tab['fov']==fov]['label'].values
    
    seg_mask = io.imread(os.path.join(seg_dir, fov+".tiff"))
    new_mask = np.zeros_like(seg_mask)
    new_mask[np.isin(seg_mask, keep_labs)] = 1
    io.imsave(os.path.join(out_dir, fov+".tiff"), new_mask, check_contrast=False)

## Distance window analysis around p24 cells

### Count number of cell types

In [ ]:
p24pos_cell_tab = pd.read_csv("../data/p24pos_cells.csv")
cell_tab = pd.read_csv(cell_tab_path)
all_fovs = np.unique(p24pos_cell_tab['fov'])

all_windows = [(0,50), (50,100), (100,150), (150,200), (200,250), (250,300), (300,350), (350,400), (400,450), (450,500)]
all_window_counts = pd.DataFrame(columns=['fov','index_cell_label','cell_meta_cluster','count','window'])

for fov in all_fovs:
    print(fov)
    one_fov_tab = p24pos_cell_tab[p24pos_cell_tab['fov']==fov]
    one_fov_labs = one_fov_tab['label'].values
    one_fov_dist = xr.load_dataarray(os.path.join(dist_mat_dir, str(fov) + '_dist_mat.xr'))
    one_fov_dist = one_fov_dist.loc[one_fov_dist.dim_0.isin(one_fov_labs),:]
    
    one_fov_all = cell_tab[cell_tab['fov']==fov][['label','cell_meta_cluster']]

    for lower_um,upper_um in all_windows:

        lower_px = lower_um * (2048/800)
        upper_px = upper_um * (2048/800)

        mask = (one_fov_dist > lower_px) & (one_fov_dist <= upper_px)
        xr_indices = np.where(mask.values)
        mask_dat = pd.DataFrame({
            'index_cell_label': one_fov_dist.dim_0.values[xr_indices[0]],
            'label': one_fov_dist.dim_1.values[xr_indices[1]]
        })

        mask_dat = pd.merge(mask_dat, one_fov_all, on='label')
        mask_counts = mask_dat.groupby(['index_cell_label','cell_meta_cluster']).size().reset_index(name='count')
        mask_counts['window'] = upper_um
        mask_counts['fov'] = fov

        all_window_counts = pd.concat([all_window_counts, mask_counts])
        
all_window_counts.to_csv("../data/p24pos_cells_dist_counts_0_500.csv", index=False)

### Get average expression around p24 cells

In [ ]:
p24pos_cell_tab = pd.read_csv("../data/p24pos_cells.csv")
cell_tab = pd.read_csv(cell_tab_path)
all_fovs = np.unique(p24pos_cell_tab['fov'])

all_windows = [(0,50), (50,100), (100,150), (150,200), (200,250), (250,300), (300,350), (350,400), (400,450), (450,500)]
all_window_exp = pd.DataFrame(columns=['fov','index_cell_label','mean_exp','window'])

markers = ['CD11c', 'CD14', 'CD163', 'CD20', 'CD21', 'CD31', 'CD3e', 'CD4', 'CD45', 'CD45RO',
           'CD56', 'CD68', 'CD69', 'CD86', 'CD8a', 'CXCR5', 'Calprotectin', 'Caspase1', 'FOXP3', 'Galectin9',
           'Glut1', 'GranzymeB', 'HH3', 'HLA1classABC', 'HLADR', 'ICOS', 'IDO', 'IFNg', 'Ki67', 'Lag3',
           'MastCellTryptase', 'NLRP3', 'PD1', 'PDL1', 'SMA', 'TCF1TCF7', 'TIGIT', 'TIM3', 'Vimentin']

for fov in all_fovs:
    print(fov)
    one_fov_tab = p24pos_cell_tab[p24pos_cell_tab['fov']==fov]
    one_fov_labs = one_fov_tab['label'].values
    one_fov_dist = xr.load_dataarray(os.path.join(dist_mat_dir, str(fov) + '_dist_mat.xr'))
    one_fov_dist = one_fov_dist.loc[one_fov_dist.dim_0.isin(one_fov_labs),:]
    
    one_fov_all = cell_tab[cell_tab['fov']==fov][['label']+markers]

    for lower_um,upper_um in all_windows:
        lower_px = lower_um * (2048/800)
        upper_px = upper_um * (2048/800)

        mask = (one_fov_dist > lower_px) & (one_fov_dist <= upper_px)
        xr_indices = np.where(mask.values)
        mask_dat = pd.DataFrame({
            'index_cell_label': one_fov_dist.dim_0.values[xr_indices[0]],
            'label': one_fov_dist.dim_1.values[xr_indices[1]]
        })

        mask_dat = pd.merge(mask_dat, one_fov_all, on='label')
        mask_dat_melt = pd.melt(mask_dat, id_vars=['index_cell_label','label'], value_vars=markers, var_name='marker', value_name='exp_value')
        mask_exp = mask_dat_melt.groupby(['index_cell_label','marker'])['exp_value'].mean().reset_index(name='mean_exp')
        mask_exp['window'] = upper_um
        mask_exp['fov'] = fov

        all_window_exp = pd.concat([all_window_exp, mask_exp])

all_window_exp.to_csv("../data/p24pos_cells_dist_mean_exp_0_500.csv", index=False)

## Look at CD8T cells around p24 cells

In [ ]:
p24pos_cell_tab_path = "../data/p24pos_cells.csv"
nimbus_tab_path = "../data/tables/nimbus_binarized.csv"
dist_mat_dir = "../data/spatial_analysis/dist_mats"

all_windows = [(0,50), (50,100), (100,150), (150,200), (200,250), (250,300), (300,350), (350,400), (400,450), (450,500)]

In [ ]:
p24pos_cell_tab = pd.read_csv(p24pos_cell_tab_path)
nimbus_tab = pd.read_csv(nimbus_tab_path)

func_markers = nimbus_tab.columns.values
func_markers = [x for x in func_markers if x not in ['fov','label','cell_meta_cluster']]
all_fovs = np.unique(p24pos_cell_tab['fov'])

all_window_counts = pd.DataFrame(columns=['index_cell_label','marker','count','total_cd8t','prop','window','fov'])
for fov in all_fovs:
    print(fov)
    one_fov_tab = p24pos_cell_tab[p24pos_cell_tab['fov']==fov]
    one_fov_labs = one_fov_tab['label'].values
    one_fov_dist = xr.load_dataarray(os.path.join(dist_mat_dir, str(fov) + '_dist_mat.xr'))
    one_fov_dist = one_fov_dist.loc[one_fov_dist.dim_0.isin(one_fov_labs),:]

    one_fov_all = nimbus_tab[nimbus_tab['fov']==fov]

    for lower_um,upper_um in all_windows:
        lower_px = lower_um * (2048/800)
        upper_px = upper_um * (2048/800)

        mask = (one_fov_dist > lower_px) & (one_fov_dist <= upper_px)
        xr_indices = np.where(mask.values)
        mask_dat = pd.DataFrame({
            'index_cell_label': one_fov_dist.dim_0.values[xr_indices[0]],
            'label': one_fov_dist.dim_1.values[xr_indices[1]]
        })

        mask_dat = pd.merge(mask_dat, one_fov_all, on='label')
        mask_dat = mask_dat[mask_dat['cell_meta_cluster']=="CD8T"]

        total_cd8t = mask_dat.groupby('index_cell_label').size().reset_index()
        total_cd8t.columns = ['index_cell_label', 'total_cd8t']

        func_counts = mask_dat.groupby('index_cell_label')[func_markers].sum().reset_index()
        func_counts = func_counts.melt(id_vars=['index_cell_label'], var_name='marker', value_name='count')
        func_counts = func_counts.merge(total_cd8t, on='index_cell_label')
        func_counts['prop'] = func_counts['count'] / func_counts['total_cd8t']
        func_counts['window'] = upper_um
        func_counts['fov'] = fov

        all_window_counts = pd.concat([all_window_counts, func_counts])

save_cols = ['fov','index_cell_label','window','marker','prop']
save_df = all_window_counts[save_cols]
save_df.to_csv("../data/p24pos_cells_dist_counts_0_500_cd8t_func.csv", index=False)